In [ ]:
using Pkg
Pkg.activate(joinpath(dirname(pwd()), "conf", "03-page_rank"))
Pkg.instantiate();

In [ ]:
using Graphs, DataFrames, SparseArrays, LinearAlgebra, Plots, FileIO
using GraphPlot, Colors
using VegaLite, VegaDatasets

This notebook is in part based on [DataScience](https://github.com/JuliaAcademy/DataScience) of the JuliaAcademy by [Huda Nassar](https://github.com/nassarhuda). 

---
---
# Data loading

We load the `flights-airport` dataset and display the first 8 dataframes

In [ ]:
flights = dataset("flights-airport") |> DataFrame;
flights[1:8,:]

All airport IDs are then extracted into the array `airports`.

In [ ]:
list_airports = vcat(flights[:,:origin],flights[:,:destination])
airports = unique(list_airports)
airports[1:5]
length(airports)

---
---
# Graph definition

We use the adjacency matrix $A$ to define the graph of airports.

In [ ]:
# build the adjacency matrix
n = length(airports)
A = spzeros(n,n)
for k = 1:size(flights, 1)
    i = findfirst(airports .== flights[k,:origin])
    j = findfirst(airports .== flights[k,:destination])
    A[i,j] = 1
end
A = max.(A,A')

In [ ]:
G = Graph(A)

---
---
# Visualisation of the abstract graph

In [ ]:
layout= x -> spring_layout(x; C=5)
gplot(G, layout = layout,
        nodefillc = colorant"steelblue",
        linetype="curve"
)

It seems like the distribution of the degrees of the nodes is not uniform. The following plot affirms this.

In [ ]:
degrees = degree(G)
p1 = plot(sort(degrees,rev=true), ylabel="log degree", legend=false, yaxis=:log, title="Grad Verteilung")
p2 = plot(sort(degrees,rev=true), ylabel="degree", legend=false)
plot(p1, p2, size=(600,200))

---
---
# Visualisation of the graph using geographical coordinates
We are gonna enrich the visualisation of the graph $G$ by pinning the nodes to their geographical coordinates in $\mathbb{R}^2$.

First we are gonna load the airport locations.

In [ ]:
all_locations = dataset("airports") |> DataFrame
k = [id in airports for id in all_locations[:,:iata]]
locations = all_locations[k,:]
locations[1:5,:]

Using the `VegaLite` package we are now visualising the graph:

In [ ]:
us10m = dataset("us-10m")
P1 = @vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:white, stroke=:lightgray},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=locations, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size={value=40}, color={value=:steelblue})+
@vlplot(
    mark={:rule, strokeWidth = 0.005,  opacity=0.1, color = "grey"},
    data=flights, 
         transform=[
                {lookup=:origin, 
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["origin_latitude", "origin_longitude"]},
                {lookup=:destination,
                 from={data=locations, key=:iata, fields=["latitude", "longitude"]},
                 as=["dest_latitude", "dest_longitude"]}],
        projection={type=:albersUsa},
        longitude="origin_longitude:q", latitude="origin_latitude:q",
        longitude2="dest_longitude:q", latitude2="dest_latitude:q")

---
---
# PageRank calculation

We are now gonna calculate the PageRank of each node (=airport) in the graph $G=(V,E)$.

The PageRank distribution is given by the eigenvector to eigenvalue 1 of the matrix
$$ P = (p_{u,v})\in \mathbb R^{\vert V\vert \times \vert V\vert} , \quad p_{u,v}=\begin{cases} 1/\mathrm{deg}(v), & \{u,v\}\in E\\ 0, &\text{else}\end{cases}.
$$

First we define the matrix.

In [ ]:
P = zeros(n, n);
for u in 1:n
    for v in 1:n
        if A[u,v] == 1
            P[u,v] = 1/degrees[v]
        end
    end
end

Now we can calculate their eigenvalues and eigenvectors.

In [ ]:
E = eigen(P)
E.values[end-2:end]

The last column of `E.vector` contains the eigenvector to eigenvalue 1.

In [ ]:
f = real.(E.vectors[:,end])
# test:
norm(P*f-f)

We are gonna create a new `DataFrame` for the PageRank

In [ ]:
PageRank =  DataFrame(airport = locations[:,:name], 
                      city = locations[:,:city],  
                      rank = f ./ sum(f),
                      longitude = locations[:,:longitude],
                      latitude = locations[:,:latitude]
)
PageRank[1:5,:]

Let's have a look at the 5 airports with largest PageRank value

In [ ]:
PageRank_sorted = sort(PageRank,[:rank],rev=true)
PageRank_sorted[1:5,:]

---
---
# Visualisation of PageRank


Additionally to previous visualisations we can further enrich the data by scaling the nodes in the visualisation with respect to their PageRank. Large node = large PageRank value

In [ ]:
layout= x -> spring_layout(x; C=20)
gplot(G, layout = layout,
        nodefillc = colorant"steelblue",
        linetype = "curve",
        nodesize = PageRank[:,:rank]
)

Most of the total PageRank value is concentrated onto some airports.

In [ ]:
p1 = plot(PageRank_sorted[:,:rank], ylabel="log degree", legend=false, yaxis=:log, title="PageRank Verteilung")
p2 = plot(PageRank_sorted[:,:rank], ylabel="degree", legend=false)
plot(p1, p2, size=(600,200))

We are gonna combine the geographical location and the PageRank value into one visualisation.

In [ ]:
P2 = @vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:white, stroke=:lightgray},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=PageRank, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size="rank", color={value=:steelblue})

---
---
# Markov Processes
Now we want to approximate PageRank using Markov Processes. Let's sample a random airport $X$ and go from there:

In [ ]:
X = rand(1:n)
locations[X,:name]

The column P_{-, X} is the distribution giving us the probablity of goging from $X$ to $Y$ as value $P_{Y, X}$.

In [ ]:
sum(P[:,X])

In [ ]:
function step(X, P)
    F = cumsum(P[:,X])
    r = rand()
    findfirst(F .> r)
end

The following cell simulates 10 steps of this random walk.

In [ ]:
ℓ = 10
walk = zeros(Int, ℓ)
walk[1] = X
for i in 2:ℓ
    walk[i] = step(walk[i-1], P)
end
locations[walk, :name]

A random walk with $\ell$ steps induces a distribution on the nodes of $G$. The function `sample` draws a random starting node and then simulates $\ell-1$ steps and stops at a node that is then returned as a sample of said distribution. In total $\ell$ nodes have been sampled from the distribution with $\ell -1$ samples being discarded - this is a sort of "warmup" of the algorithm.

In [ ]:
function sample(ℓ, P)
    X = rand(1:n)
    for i in 2:ℓ
        X = step(X, P)
    end
   X
end

Now lets sample with $\ell = 1000$ to get a random airport like this:

In [ ]:
X = sample(1000, P)
locations[X, :name]

Finally this gives us a methode to approximatley draw from the PageRank distribution without explicitly calculating the eigenvalues!

In [ ]:
Ω = [sample(20, P) for _ in 1:1e4];
p = [count(Ω .== i) for i in 1:n]
PageRankApprox =  DataFrame(airport = locations[:, :name], 
                      city = locations[:, :city],  
                      rank =  p./sum(p),
                      longitude = locations[:, :longitude] .+ 0.5, #zur Visualisuerung
                      latitude = locations[:, :latitude] .+ 0.5
)
PageRankApprox[1:5,:]

Let's visualise the approximation: PageRank = blue, approximation = brown

In [ ]:
@vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:white, stroke=:lightgray},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=PageRankApprox, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size="rank", color={value=:tan}) +
@vlplot(
    :circle,
    data=PageRank, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size="rank", color={value=:steelblue})

Finally, we plot Page-Rank against the approximated Page-Rank.

In [ ]:
plot(sort(PageRankApprox[:,"rank"]), 
                    linewidth = 4, 
                    color = :tan,
                    label = "Approximation of Page-Rank by Markov process",
                    legend = :left,
                    xlabel = "vertices",
                    ylabel = "centrality measure")
plot!(sort(PageRank[:,"rank"]), 
                    linewidth = 4, 
                    color = :steelblue,
                    label = "Page-Rank",
                    linestyle = :dash)

---
# Metropolis-Hastings

Let $u\in V$ be an airport then $f(u)$ is the number of connections from and to the airport.


In [ ]:
f = zeros(n)
for k = 1:n
    i = findall(flights[:,:origin] .== airports[k])
    f[k] += sum(flights[i, :count])
    j = findall(flights[:,:destination] .== airports[k])
    f[k] += sum(flights[j, :count])
end
plot(sort(f), label = "f", lw = 3, size = (600,300))

We want to use Metropolis-Hastings to sample from
$$\pi(u) = \frac{f(u)}{\sum_{v\in V} f(v)},\quad v\in V$$

We define the transition matrix `Q` as

$$q_{uv}=
\begin{cases}
\tfrac{1}{d}\min\{1, \tfrac{\pi(u)}{\pi(v)}\}, &\text{ if } u\neq v\text{ and }\{u,v\}\in E\\
0, &\text{ if } u\neq v\text{ and } \{u,v\}\not\in E\\
1- \sum_{i\neq v} q_{iv}, &\text{ if } u=v.
\end{cases}$$

with $d>\max_{v\in V}\mathrm{deg}(v)$.

In [ ]:
d = maximum(degrees)+1
Q = (1/d) .* [A[u,v] .* min(1, f[u]/f[v]) for (u,v) in Iterators.product(1:n,1:n)]
Q = Q + diagm(ones(n) - Q' * ones(n));

We can start algorithm now by using 2000 steps and 1000 samples.

In [ ]:
Ω₂ = [sample(2000, Q) for _ in 1:1e3];
q = [count(Ω₂  .== i) for i in 1:n]
Count = DataFrame(airport = locations[:, :name], 
                      city = locations[:, :city],  
                      rank =  q./sum(q),
                      longitude = locations[:, :longitude],
                      latitude = locations[:, :latitude])
Count[1:5,:]

In [ ]:
histogram(Count[:, :rank], label = "Count", lw = 3, size = (600,300))

In [ ]:
@vlplot(width=800, height=500) +
@vlplot(
    mark={:geoshape, fill=:white, stroke=:lightgray},
    data={values=us10m, format={type=:topojson,feature=:states}},
    projection={type=:albersUsa}) +
@vlplot(
    :circle,
    data=Count, projection={type=:albersUsa}, 
    longitude="longitude:q", latitude="latitude:q", 
    size="rank", color={value=:indianred})